# This notebook will demonstrate a few features to get you started with creating your own notebook for custom analyses. 

---
## Let's first connect to the appropriate database. The custom `sfmta` library helps us do this.

In [8]:
import sfmta
import os
import psycopg2
import datetime
import pandas as pd

os.environ['SHUTTLE_DB_USER'] = "postgres"
os.environ['SHUTTLE_DB_PASSWORD'] =''
os.environ['SHUTTLE_DB_HOST'] ="localhost"
conn = sfmta.db_connect()

Connection Created


## Now we'll execute or first `SQL` query to retrieve all points for a single shuttle between two dates and times (e.g., from 7:30am on 9/19/2017 to 7:30am on 9/20/2017).

## After retrieving the relevant data, we can then visualize it on a map or graph it in a chart.

In [11]:
plate = 'CP96872'

# 2017-09-19 07:30:00
start = datetime.datetime(year=2017, month=9, day=19, hour=7, minute=30)

#2017-09-20 07:30:00
end = datetime.datetime(year=2017, month=9, day=20, hour=7, minute=30)

def get_points_for_shuttle(connection, plate, start, end):
    cursor = connection.cursor()
    query = '''SELECT local_timestamp, location[0], location[1] 
               FROM shuttle_locations l LEFT OUTER JOIN shuttles s 
               ON l.shuttle_id=s.id WHERE s.vehicle_license_plate=%(plate)s
               AND local_timestamp BETWEEN %(start)s::timestamp
                 AND %(end)s::timestamp
               ORDER BY local_timestamp;'''

    cursor.execute(query, {'plate': plate, 'start':start, 'end':end,} )
    results = cursor.fetchall()
    return pd.DataFrame.from_records(results, columns=['time', 'lat', 'lng']).set_index('time')

dataframe = get_points_for_shuttle(connection=conn, plate=plate, start=start, end=end)

# Retrive top 5 rows for a quick view of the data
dataframe.head(n=5)

,lat,lng
time,,
2017-09-19 08:11:22,-122.396966,37.746787
2017-09-19 08:11:28,-122.396710,37.746774
2017-09-19 08:11:34,-122.396672,37.746752
2017-09-19 08:11:40,-122.396634,37.746726
2017-09-19 08:11:46,-122.396557,37.746698
